<a href="https://colab.research.google.com/github/phsieg/gan_artwork_open/blob/main/tf_gan_tutorial_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook is for testing the tutorial at https://www.tensorflow.org/tutorials/generative/dcgan

# Find Local Modules Help

In [6]:
help('modules')


Please wait a moment while I gather a list of all available modules...



/usr/local/lib/python3.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/usr/local/lib/python3.7/dist-packages/llvmlite/llvmpy/__init__.py:4: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  "The module `llvmlite.llvmpy` is deprecated and will be removed in the "
/usr/local/lib/python3.7/dist-packages/llvmlite/llvmpy/core.py:9: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  "The module `llvmlite.llvmpy.core` is deprecated and will be removed in "
/usr/local/lib/python3.7/dist-packages/caffe2/__init__.py:5: UserWarning: Caffe2 support is not fully enabled in this PyTorch build. Please enable Caffe2 by building PyTorch from sour

/usr/local/lib/python3.7/dist-packages/nbconvert/filters/strings.py:22: DeprecationWarning:

defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.

/usr/local/lib/python3.7/dist-packages/nibabel/quaternions.py:31: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/lib/python3.7/pkgutil.py:92: UserWarning:

The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:21: UserWarning:

The twython library has not been installed. Some functionality fro

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Cython              collections         kapre               resource
IPython             colorcet            keras               rlcompleter
OpenGL              colorlover          keras_preprocessing rmagic
PIL                 colorsys            keyword             rpy2
ScreenResolution    community           kiwisolver          rsa
__future__          compileall          korean_lunar_calendar runpy
_abc                concurrent          langcodes           samples
_ast                configparser        lib2to3             sched
_asyncio            cons                libfuturize         scipy
_bisect             contextlib          libpasteurize       scs
_blake2             contextlib2         librosa             seaborn
_bootlocale         contextvars         lightgbm            secrets
_bz2                convertdate         linecache           select
_cffi_backend       copy                llvmlite            selector

/usr/lib/python3.7/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    


# Tensorflow

In [7]:
import tensorflow as tf
# for drawing generated images by plotting
import matplotlib.pyplot as plt 

# Read images

In [2]:
print("hello world")

hello world
